In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.applications import imagenet_utils
from sklearn.metrics import confusion_matrix
#from tflite_model_maker import ImageClassifierDataLoader

import itertools
import os
import shutil
import random
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
mobile = tf.keras.applications.mobilenet.MobileNet()

17235968/17225924 [==============================] - 0s 0us/step


In [3]:
from IPython.display import Image

In [5]:
train_path = 'drive/MyDrive/cup01/train'
valid_path = 'drive/MyDrive/cup01/valid'
test_path = 'drive/MyDrive/cup01/test'

BATCH_SIZE = 10

train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=train_path, target_size=(224,224), batch_size=BATCH_SIZE)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=valid_path, target_size=(224,224), batch_size=BATCH_SIZE)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=test_path, target_size=(224,224), batch_size=BATCH_SIZE, shuffle=False)

Found 123 images belonging to 10 classes.
Found 30 images belonging to 10 classes.
Found 10 images belonging to 10 classes.


In [6]:
test_path

'drive/MyDrive/cup01/test'

In [7]:
mobile = tf.keras.applications.mobilenet.MobileNet()
mobile.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 112, 112, 32

In [8]:
x = mobile.layers[-6].output
output = Dense(units=10, activation='softmax')(x)

In [9]:
model = Model(inputs=mobile.input, outputs=output)

In [10]:
for layer in model.layers[:-23]:
    layer.trainable = False

In [11]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 112, 112, 32)      0     

In [12]:
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [13]:
model.fit(x=train_batches,
            steps_per_epoch=len(train_batches),
            validation_data=valid_batches,
            validation_steps=len(valid_batches),
            epochs=30,
            verbose=2
)

Epoch 1/30
13/13 - 94s - loss: 1.2357 - accuracy: 0.5935 - val_loss: 1.7516 - val_accuracy: 0.4333
Epoch 2/30
13/13 - 27s - loss: 0.0853 - accuracy: 0.9837 - val_loss: 1.8128 - val_accuracy: 0.2000
Epoch 3/30
13/13 - 27s - loss: 0.0498 - accuracy: 0.9919 - val_loss: 1.1478 - val_accuracy: 0.4333
Epoch 4/30
13/13 - 27s - loss: 0.0166 - accuracy: 1.0000 - val_loss: 0.7386 - val_accuracy: 0.7000
Epoch 5/30
13/13 - 27s - loss: 0.0159 - accuracy: 1.0000 - val_loss: 0.4497 - val_accuracy: 0.8667
Epoch 6/30
13/13 - 27s - loss: 0.0154 - accuracy: 1.0000 - val_loss: 0.2902 - val_accuracy: 0.9667
Epoch 7/30
13/13 - 27s - loss: 0.0074 - accuracy: 1.0000 - val_loss: 0.2007 - val_accuracy: 0.9333
Epoch 8/30
13/13 - 27s - loss: 0.0074 - accuracy: 1.0000 - val_loss: 0.1532 - val_accuracy: 0.9667
Epoch 9/30
13/13 - 27s - loss: 0.0052 - accuracy: 1.0000 - val_loss: 0.1302 - val_accuracy: 0.9667
Epoch 10/30
13/13 - 27s - loss: 0.0116 - accuracy: 1.0000 - val_loss: 0.1347 - val_accuracy: 0.9333
Epoch 11/